In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_selection import VarianceThreshold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import re

In [2]:
df = pd.read_csv('../data/test_processed.csv', index_col='Unnamed: 0')

In [3]:
df.head()

,season,Wk,Home,Away,result,ht_win,ht_draw,ht_lose,ht_goals_for,ht_goals_against,...,at_home_goals_against,at_points,at_position,at_goal_diff,ht_last_5_games_home,ht_last_5_games_away,ht_last_5_games,at_last_5_games_home,at_last_5_games_away,at_last_5_games
0,15,1.0,Manchester Utd,Tottenham,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
1,15,1.0,Leicester City,Sunderland,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
2,15,1.0,Everton,Watford,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,1.0,Norwich City,Crystal Palace,-1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0
4,15,1.0,Bournemouth,Aston Villa,-1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0


In [4]:
X = df.drop(columns='result')
y = df['result']

In [5]:
X.columns

Index(['season', 'Wk', 'Home', 'Away', 'ht_win', 'ht_draw', 'ht_lose',
       'ht_goals_for', 'ht_goals_against', 'ht_home_win', 'ht_home_draw',
       'ht_home_lose', 'ht_home_goals_for', 'ht_home_goals_against',
       'ht_points', 'ht_position', 'ht_goal_diff', 'at_win', 'at_draw',
       'at_lose', 'at_goals_for', 'at_goals_against', 'at_home_win',
       'at_home_draw', 'at_home_lose', 'at_home_goals_for',
       'at_home_goals_against', 'at_points', 'at_position', 'at_goal_diff',
       'ht_last_5_games_home', 'ht_last_5_games_away', 'ht_last_5_games',
       'at_last_5_games_home', 'at_last_5_games_away', 'at_last_5_games'],
      dtype='object')

In [6]:
cat_cols = ['Home', 'Away']
num_cols = [col for col in X.columns if col not in cat_cols]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

In [8]:
full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OrdinalEncoder(), cat_cols)
])

In [9]:
X_train = full_pipeline.fit_transform(X_train)
X_test = full_pipeline.transform(X_test)

In [10]:
log_reg = LogisticRegression(random_state=0, n_jobs=-1, solver='newton-cg')
rf = RandomForestClassifier(random_state=0, n_jobs=-1)
ad = AdaBoostClassifier(random_state=0)
xgb = XGBClassifier(random_state=0, n_jobs=-1)
lgb = LGBMClassifier(random_state=0, n_jobs=-1)

In [11]:
models = [log_reg, rf, ad, xgb, lgb]

In [12]:
for model in models:
    name = re.search(r'^([A-Z])\w+', str(model)).group(0)
    model.fit(X_train, y_train)
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    cv = np.mean(cross_val_score(model, X_train, y_train, n_jobs=-1, cv=10))
    
    print(f'{name} Train accuracy: {train_score*100:.2f}  Test accuracy: {test_score*100:.2f}  10-fold cv accuracy: {cv*100:.2f}')

LogisticRegression Train accuracy: 71.85  Test accuracy: 68.78  10-fold cv accuracy: 68.88
RandomForestClassifier Train accuracy: 100.00  Test accuracy: 65.47  10-fold cv accuracy: 63.69
AdaBoostClassifier Train accuracy: 69.78  Test accuracy: 62.98  10-fold cv accuracy: 63.69
XGBClassifier Train accuracy: 100.00  Test accuracy: 69.06  10-fold cv accuracy: 65.35
LGBMClassifier Train accuracy: 100.00  Test accuracy: 67.40  10-fold cv accuracy: 65.90


Best models are Logistic Regression, XGBoost and LightGBM. Only they will be evaluated from now on.

## Log reg

In [51]:
params = {'tol': [1e-4, 1e-3, 1e-2, 1],
          'C': [.001, .01, .05, .1, .5, 1],
          'fit_intercept': [True, False],
          'max_iter': [50, 100, 250],
          'multi_class': ['auto', 'multinomial']}

In [52]:
log_reg_grid = GridSearchCV(log_reg, params, n_jobs=-1, verbose=20)
log_reg_grid.fit(X_train, y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 147 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 148 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 150 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 151 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 153 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 155 tasks      | elapsed:    3.6s
[Paralle

[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 275 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 279 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 281 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 283 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 285 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 286 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 287 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 288 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 289 tasks      | elapsed:    5.7s
[Paralle

[Parallel(n_jobs=-1)]: Done 409 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 411 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 412 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 413 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 414 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 415 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 416 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 417 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 419 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 420 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 421 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 422 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 423 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 424 tasks      | elapsed:    7.9s
[Paralle

...

[Parallel(n_jobs=-1)]: Done 1392 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 1393 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 1394 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 1395 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 1396 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 1397 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 1398 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 1399 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 1400 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 1401 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 1402 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 1403 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 1404 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 1405 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 1406 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 1407 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 1523 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 1524 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 1525 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 1526 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 1527 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 1528 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 1529 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 1530 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 1531 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 1532 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 1533 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 1534 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 1535 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 1536 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 1537 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 1538 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 1661 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 1662 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 1663 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 1664 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 1665 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 1666 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 1667 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 1668 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 1669 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 1670 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 1671 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 1672 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 1673 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done 1674 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done 1675 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done 1676 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 1801 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done 1802 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 1803 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 1804 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 1805 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 1806 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 1807 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 1808 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 1809 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 1810 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 1811 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 1812 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 1813 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 1814 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 1815 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 1816 tasks      | elapsed: 

GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=-1, penalty='l2',
                                          random_state=0, solver='newton-cg',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.05, 0.1, 0.5, 1],
                         'fit_intercept': [True, False],
                         'max_iter': [50, 100, 250, 500],
                         'multi_class': ['auto', 'multinomial'],
                         'tol': [0.0001, 0.001, 0.01, 1]},
             pre_dispatch='2*n_jobs', refit=True,

In [53]:
log_reg_grid.best_score_

0.694330032215726

In [54]:
log_reg_grid.best_params_

{'C': 0.5,
 'fit_intercept': True,
 'max_iter': 50,
 'multi_class': 'auto',
 'tol': 1}

## XGB

In [17]:
params = {'eta':[.01, .05, .1, .3, .5, 1],
         'gamma': [0, 1, 10, 100],
         'max_depth': [3, 6, 9],
         'min_child_weight': [1, 10, 100]}

In [18]:
xgb_grid = GridSearchCV(xgb, params, n_jobs=6, verbose=20)
xgb_grid.fit(X_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done   2 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done   3 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done   4 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done   5 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done   7 tasks      | elapsed:    1.4s
[Parallel(n_jobs=6)]: Done   8 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done   9 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done  10 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done  11 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done  12 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    2.2s
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    2.2s
[Parallel(n_jobs=6)]: Done  15 tasks      | elapsed:    2.2s
[Parallel(

[Parallel(n_jobs=6)]: Done 135 tasks      | elapsed:   26.9s
[Parallel(n_jobs=6)]: Done 136 tasks      | elapsed:   27.3s
[Parallel(n_jobs=6)]: Done 137 tasks      | elapsed:   27.3s
[Parallel(n_jobs=6)]: Done 138 tasks      | elapsed:   27.3s
[Parallel(n_jobs=6)]: Done 139 tasks      | elapsed:   27.4s
[Parallel(n_jobs=6)]: Done 140 tasks      | elapsed:   27.5s
[Parallel(n_jobs=6)]: Done 141 tasks      | elapsed:   27.7s
[Parallel(n_jobs=6)]: Done 142 tasks      | elapsed:   28.1s
[Parallel(n_jobs=6)]: Done 143 tasks      | elapsed:   28.1s
[Parallel(n_jobs=6)]: Done 144 tasks      | elapsed:   28.2s
[Parallel(n_jobs=6)]: Done 145 tasks      | elapsed:   28.2s
[Parallel(n_jobs=6)]: Done 146 tasks      | elapsed:   28.2s
[Parallel(n_jobs=6)]: Done 147 tasks      | elapsed:   28.5s
[Parallel(n_jobs=6)]: Done 148 tasks      | elapsed:   28.9s
[Parallel(n_jobs=6)]: Done 149 tasks      | elapsed:   28.9s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   28.9s
[Parallel(n_jobs=6)]: Do

[Parallel(n_jobs=6)]: Done 270 tasks      | elapsed:   53.5s
[Parallel(n_jobs=6)]: Done 271 tasks      | elapsed:   53.8s
[Parallel(n_jobs=6)]: Done 272 tasks      | elapsed:   53.9s
[Parallel(n_jobs=6)]: Done 273 tasks      | elapsed:   54.0s
[Parallel(n_jobs=6)]: Done 274 tasks      | elapsed:   54.0s
[Parallel(n_jobs=6)]: Done 275 tasks      | elapsed:   54.0s
[Parallel(n_jobs=6)]: Done 276 tasks      | elapsed:   54.3s
[Parallel(n_jobs=6)]: Done 277 tasks      | elapsed:   54.5s
[Parallel(n_jobs=6)]: Done 278 tasks      | elapsed:   54.6s
[Parallel(n_jobs=6)]: Done 279 tasks      | elapsed:   54.7s
[Parallel(n_jobs=6)]: Done 280 tasks      | elapsed:   54.8s
[Parallel(n_jobs=6)]: Done 281 tasks      | elapsed:   54.8s
[Parallel(n_jobs=6)]: Done 282 tasks      | elapsed:   55.0s
[Parallel(n_jobs=6)]: Done 283 tasks      | elapsed:   55.3s
[Parallel(n_jobs=6)]: Done 284 tasks      | elapsed:   55.4s
[Parallel(n_jobs=6)]: Done 285 tasks      | elapsed:   55.5s
[Parallel(n_jobs=6)]: Do

[Parallel(n_jobs=6)]: Done 405 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done 406 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done 407 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 408 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 409 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 410 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 411 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 412 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 413 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 414 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 415 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 416 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 417 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 418 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 419 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 420 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Do

[Parallel(n_jobs=6)]: Done 540 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 541 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 542 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 543 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 544 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 545 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 546 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 547 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 548 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 549 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 550 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 551 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 552 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 553 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 554 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done 555 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Do

[Parallel(n_jobs=6)]: Done 676 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 677 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 678 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 679 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 680 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 681 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 682 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 683 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 684 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 685 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 686 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 687 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 688 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 689 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 690 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 691 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Do

[Parallel(n_jobs=6)]: Done 811 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 812 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 813 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 814 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 815 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 816 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 817 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 818 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 819 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 820 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 821 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 822 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 823 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 824 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 825 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Done 826 tasks      | elapsed:  2.7min
[Parallel(n_jobs=6)]: Do

[Parallel(n_jobs=6)]: Done 947 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 948 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 949 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 950 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 951 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 952 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 953 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 954 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 955 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 956 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 957 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 958 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 959 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 960 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 961 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 962 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Do

GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster=None,
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints=None,
                                     n_estimators=100, n_jo...
                                     objective='multi:softprob', random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=None, subsample=1,
                                     tree_method=None,
         

In [19]:
xgb_grid.best_score_

0.6659634888438134

In [20]:
xgb_grid.best_params_

{'eta': 0.01, 'gamma': 1, 'max_depth': 3, 'min_child_weight': 1}

## LGB

In [33]:
params = {
    'max_depth': [-1, 3, 5],
    'min_sum_hessian_in_leaf': [ 1e-4, 1e-3, 1e-2, 1],
    'early_stopping_round': [0, 100, 1000, 2500],
    'n_estimators': [100, 250, 500, 1000],
    'learing_rate': [.01, .05, .1, 1],
    'num_leaves': [20, 31, 50]
}

In [34]:
lgb_grid = GridSearchCV(lgb, params, n_jobs=6, verbose=20)
lgb_grid.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='multi_error')

Fitting 5 folds for each of 2304 candidates, totalling 11520 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Batch computation too fast (0.0300s.) Setting batch_size=2.
[Parallel(n_jobs=6)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  11 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Batch computation too fast (

[Parallel(n_jobs=6)]: Done 4340 tasks      | elapsed:    4.7s
[Parallel(n_jobs=6)]: Done 4404 tasks      | elapsed:    5.0s
[Parallel(n_jobs=6)]: Done 4468 tasks      | elapsed:    5.0s
[Parallel(n_jobs=6)]: Done 4532 tasks      | elapsed:    5.0s
[Parallel(n_jobs=6)]: Done 4596 tasks      | elapsed:    5.1s
[Parallel(n_jobs=6)]: Done 4660 tasks      | elapsed:    5.1s
[Parallel(n_jobs=6)]: Done 4724 tasks      | elapsed:    5.3s
[Parallel(n_jobs=6)]: Done 4788 tasks      | elapsed:    5.4s
[Parallel(n_jobs=6)]: Done 4852 tasks      | elapsed:    5.6s
[Parallel(n_jobs=6)]: Done 4916 tasks      | elapsed:    5.6s
[Parallel(n_jobs=6)]: Done 4980 tasks      | elapsed:    5.6s
[Parallel(n_jobs=6)]: Done 5044 tasks      | elapsed:    5.8s
[Parallel(n_jobs=6)]: Done 5108 tasks      | elapsed:    6.0s
[Parallel(n_jobs=6)]: Done 5172 tasks      | elapsed:    6.1s
[Parallel(n_jobs=6)]: Done 5236 tasks      | elapsed:    6.2s
[Parallel(n_jobs=6)]: Done 5300 tasks      | elapsed:    6.3s
[Paralle

[Parallel(n_jobs=6)]: Done 7602 tasks      | elapsed:   26.2s
[Parallel(n_jobs=6)]: Done 7603 tasks      | elapsed:   26.8s
[Parallel(n_jobs=6)]: Done 7604 tasks      | elapsed:   26.9s
[Parallel(n_jobs=6)]: Done 7605 tasks      | elapsed:   27.0s
[Parallel(n_jobs=6)]: Done 7606 tasks      | elapsed:   28.2s
[Parallel(n_jobs=6)]: Done 7607 tasks      | elapsed:   28.4s
[Parallel(n_jobs=6)]: Done 7608 tasks      | elapsed:   28.5s
[Parallel(n_jobs=6)]: Done 7609 tasks      | elapsed:   29.2s
[Parallel(n_jobs=6)]: Done 7610 tasks      | elapsed:   29.4s
[Parallel(n_jobs=6)]: Done 7611 tasks      | elapsed:   29.4s
[Parallel(n_jobs=6)]: Done 7612 tasks      | elapsed:   30.5s
[Parallel(n_jobs=6)]: Done 7613 tasks      | elapsed:   30.5s
[Parallel(n_jobs=6)]: Done 7614 tasks      | elapsed:   30.6s
[Parallel(n_jobs=6)]: Done 7615 tasks      | elapsed:   31.2s
[Parallel(n_jobs=6)]: Done 7616 tasks      | elapsed:   31.3s
[Parallel(n_jobs=6)]: Done 7617 tasks      | elapsed:   31.4s
[Paralle

[Parallel(n_jobs=6)]: Done 7735 tasks      | elapsed:   56.5s
[Parallel(n_jobs=6)]: Done 7736 tasks      | elapsed:   56.6s
[Parallel(n_jobs=6)]: Done 7737 tasks      | elapsed:   56.9s
[Parallel(n_jobs=6)]: Done 7738 tasks      | elapsed:   56.9s
[Parallel(n_jobs=6)]: Done 7739 tasks      | elapsed:   57.2s
[Parallel(n_jobs=6)]: Done 7740 tasks      | elapsed:   57.3s
[Parallel(n_jobs=6)]: Done 7741 tasks      | elapsed:   57.6s
[Parallel(n_jobs=6)]: Done 7742 tasks      | elapsed:   57.6s
[Parallel(n_jobs=6)]: Done 7743 tasks      | elapsed:   57.6s
[Parallel(n_jobs=6)]: Done 7744 tasks      | elapsed:   57.9s
[Parallel(n_jobs=6)]: Done 7745 tasks      | elapsed:   58.0s
[Parallel(n_jobs=6)]: Done 7746 tasks      | elapsed:   58.0s
[Parallel(n_jobs=6)]: Done 7747 tasks      | elapsed:   58.3s
[Parallel(n_jobs=6)]: Done 7748 tasks      | elapsed:   58.3s
[Parallel(n_jobs=6)]: Done 7749 tasks      | elapsed:   58.4s
[Parallel(n_jobs=6)]: Done 7750 tasks      | elapsed:   58.4s
[Paralle

...

[1]	valid_0's multi_error: 0.533149	valid_0's multi_logloss: 1.02096
Training until validation scores don't improve for 1000 rounds
[2]	valid_0's multi_error: 0.475138	valid_0's multi_logloss: 0.98697
[3]	valid_0's multi_error: 0.39779	valid_0's multi_logloss: 0.95818
[4]	valid_0's multi_error: 0.350829	valid_0's multi_logloss: 0.929467
[5]	valid_0's multi_error: 0.345304	valid_0's multi_logloss: 0.907672
[6]	valid_0's multi_error: 0.348066	valid_0's multi_logloss: 0.887797
[7]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 0.867152
[8]	valid_0's multi_error: 0.337017	valid_0's multi_logloss: 0.854949
[9]	valid_0's multi_error: 0.342541	valid_0's multi_logloss: 0.841762
[10]	valid_0's multi_error: 0.339779	valid_0's multi_logloss: 0.830465
[11]	valid_0's multi_error: 0.325967	valid_0's multi_logloss: 0.819475
[12]	valid_0's multi_error: 0.325967	valid_0's multi_logloss: 0.810525
[13]	valid_0's multi_error: 0.331492	valid_0's multi_logloss: 0.804079
[14]	valid_0's multi_error:

[143]	valid_0's multi_error: 0.31768	valid_0's multi_logloss: 0.904585
[144]	valid_0's multi_error: 0.312155	valid_0's multi_logloss: 0.905626
[145]	valid_0's multi_error: 0.314917	valid_0's multi_logloss: 0.907421
[146]	valid_0's multi_error: 0.314917	valid_0's multi_logloss: 0.912813
[147]	valid_0's multi_error: 0.314917	valid_0's multi_logloss: 0.915216
[148]	valid_0's multi_error: 0.314917	valid_0's multi_logloss: 0.920279
[149]	valid_0's multi_error: 0.31768	valid_0's multi_logloss: 0.92198
[150]	valid_0's multi_error: 0.31768	valid_0's multi_logloss: 0.925679
[151]	valid_0's multi_error: 0.320442	valid_0's multi_logloss: 0.928553
[152]	valid_0's multi_error: 0.323204	valid_0's multi_logloss: 0.932042
[153]	valid_0's multi_error: 0.325967	valid_0's multi_logloss: 0.934851
[154]	valid_0's multi_error: 0.320442	valid_0's multi_logloss: 0.93759
[155]	valid_0's multi_error: 0.325967	valid_0's multi_logloss: 0.940716
[156]	valid_0's multi_error: 0.325967	valid_0's multi_logloss: 0.9426

[276]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.26939
[277]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.27298
[278]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.27627
[279]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.27917
[280]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.28238
[281]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.28575
[282]	valid_0's multi_error: 0.337017	valid_0's multi_logloss: 1.28746
[283]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.29005
[284]	valid_0's multi_error: 0.337017	valid_0's multi_logloss: 1.2922
[285]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.29526
[286]	valid_0's multi_error: 0.337017	valid_0's multi_logloss: 1.29838
[287]	valid_0's multi_error: 0.337017	valid_0's multi_logloss: 1.30049
[288]	valid_0's multi_error: 0.337017	valid_0's multi_logloss: 1.30345
[289]	valid_0's multi_error: 0.337017	valid_0's multi_logloss: 1.30606
[290]	v

[420]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.64317
[421]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.64419
[422]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.64532
[423]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.64737
[424]	valid_0's multi_error: 0.337017	valid_0's multi_logloss: 1.64817
[425]	valid_0's multi_error: 0.337017	valid_0's multi_logloss: 1.64975
[426]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.6509
[427]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.65331
[428]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.65499
[429]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.65603
[430]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.65816
[431]	valid_0's multi_error: 0.337017	valid_0's multi_logloss: 1.66071
[432]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.66168
[433]	valid_0's multi_error: 0.334254	valid_0's multi_logloss: 1.66311
[434]	v

GridSearchCV(cv=None, error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=0, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='deprecated', n_jobs=6,
             param_grid={'early_stopping_round': [0, 100, 1000, 2500],
                         '

In [35]:
lgb_grid.best_score_

0.6652690609712446

In [36]:
lgb_grid.best_params_

{'early_stopping_round': 1000,
 'learing_rate': 1,
 'max_depth': -1,
 'min_sum_hessian_in_leaf': 0.001,
 'n_estimators': 500,
 'num_leaves': 50}

In [37]:
log_reg_grid.predict_proba(X_test)

array([[9.13351488e-01, 7.82289655e-02, 8.41954694e-03],
       [9.72973518e-01, 2.69136561e-02, 1.12825705e-04],
       [7.17490049e-01, 2.26496096e-01, 5.60138553e-02],
       ...,
       [3.93544919e-02, 1.72178706e-01, 7.88466802e-01],
       [2.67532734e-01, 4.38301992e-01, 2.94165274e-01],
       [1.88720870e-02, 1.61052172e-01, 8.20075741e-01]])

In [41]:
log_reg_grid.predict(X_test)

array([-1, -1, -1,  1,  1,  0,  1,  1, -1,  1,  1, -1,  1, -1,  1, -1,  1,
        1,  1, -1,  1, -1, -1,  1,  1,  1,  0, -1,  1, -1,  1,  1,  1,  1,
        1,  1,  1,  1,  0,  1, -1,  1,  1, -1,  1,  1,  0,  1, -1,  1, -1,
        1,  1,  1,  0,  0, -1,  1,  0,  1, -1, -1, -1, -1, -1,  0,  1, -1,
       -1,  1, -1, -1,  1,  1,  0,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  0,  0,  1,  1,  1, -1,
       -1,  1, -1, -1,  1, -1, -1,  0,  0,  0,  1, -1,  0,  1,  1,  1, -1,
       -1,  1,  1,  0,  1,  1, -1,  1,  1,  0,  0, -1, -1,  0, -1, -1, -1,
        1, -1, -1,  1, -1,  1, -1,  0,  1,  0,  1,  1,  1,  0,  1,  1,  1,
       -1, -1, -1,  1,  1,  1, -1,  0,  1,  1,  1, -1,  1, -1, -1,  1,  1,
        1,  1,  1, -1,  1,  1,  0,  1, -1,  1,  1,  1,  1, -1, -1,  1, -1,
        1, -1,  0, -1,  1,  1, -1,  1,  1,  1,  0, -1, -1,  0,  1, -1, -1,
        1,  1, -1, -1, -1,  0,  0,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,
        0,  1,  1,  1, -1

In [40]:
y_test

1714    0
1142   -1
1739   -1
542     0
1305    1
       ..
171     1
686     1
1441   -1
1525    0
270     0
Name: result, Length: 362, dtype: int64

In [48]:
log_reg_grid.best_params_

{'C': 0.5,
 'fit_intercept': True,
 'max_iter': 50,
 'multi_class': 'auto',
 'tol': 1}

In [57]:
final_model = LogisticRegression(random_state=0, **log_reg_grid.best_params_,solver='newton-cg')

In [58]:
final_model.fit(X_train, y_train)

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=50,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='newton-cg', tol=1, verbose=0,
                   warm_start=False)

In [69]:
num_cols

['season',
 'Wk',
 'ht_win',
 'ht_draw',
 'ht_lose',
 'ht_goals_for',
 'ht_goals_against',
 'ht_home_win',
 'ht_home_draw',
 'ht_home_lose',
 'ht_home_goals_for',
 'ht_home_goals_against',
 'ht_points',
 'ht_position',
 'ht_goal_diff',
 'at_win',
 'at_draw',
 'at_lose',
 'at_goals_for',
 'at_goals_against',
 'at_home_win',
 'at_home_draw',
 'at_home_lose',
 'at_home_goals_for',
 'at_home_goals_against',
 'at_points',
 'at_position',
 'at_goal_diff',
 'ht_last_5_games_home',
 'ht_last_5_games_away',
 'ht_last_5_games',
 'at_last_5_games_home',
 'at_last_5_games_away',
 'at_last_5_games']